In [1]:
import os
from six.moves.urllib.request import urlopen

import numpy as np
import tensorflow as tf

/Users/andyxie/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# 介绍
Iris是两种花的数据集，根据性状不同，可以将花进行分类。这里使用它作为Estimator的教学。

# 获取数据
首先，我们将网上的数据进行下载然后存到本地，之后，我们使用tf的内置函数倒入csv数据。我们还会讨论使用pandas和numpy进行同一作业的方法。

In [3]:
IRIS_TRAINING = "../input/iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "../input/iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
#倒入数据并存在本地
if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
          f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
          f.write(raw)

In [4]:
# 使用tf内建方法倒入数据
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [5]:
training_set.data.shape # 数据共有120行，每行四个参数

(120, 4)

In [7]:
# 描述输入值格式
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

# 建立模型
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[10, 20, 10],
                                          n_classes=3,
                                          model_dir="/tmp/iris_model")
 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/iris_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a15d646d8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# 将数据装入input_fn
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": np.array(training_set.data)},
  y=np.array(training_set.target),
  num_epochs=None,
  shuffle=True)  
# 进行训练
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 6.04717, step = 2001
INFO:tensorflow:global_step/sec: 386.536
INFO:tensorflow:loss = 4.89154, step = 2101 (0.260 sec)
INFO:tensorflow:global_step/sec: 508.083
INFO:tensorflow:loss = 4.41077, step = 2201 (0.198 sec)
INFO:tensorflow:global_step/sec: 450.096
INFO:tensorflow:loss = 7.85771, step = 2301 (0.226 sec)
INFO:tensorflow:global_step/sec: 462.783
INFO:tensorflow:loss = 4.06811, step = 2401 (0.213 sec)
INFO:tensorflow:global_step/sec: 420.907
INFO:tensorflow:loss = 3.07601, step = 2501 (0.240 sec)
INFO:tensorflow:global_step/sec: 424.849
INFO:tensorflow:loss = 9.86563, step = 2601 (0.238 sec)
INFO:tensorflow:global_step/sec: 341.628
INFO:tensorflow:loss = 3.13311, step = 2701 (0.287 sec)
INFO:tensorflow:global_step/sec: 501.902
INFO:tensorflow:loss = 5.30232, step = 28

# 验证和预测
验证和预测的语法基本可以参考训练。

In [9]:
# 将测试数据放入函数
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False)

# 使用evaluate函数即可得到包括loss，accuracy在内的很多常用量
# 但是怎样进行自定义呢？？
classifier.evaluate(input_fn=test_input_fn)


INFO:tensorflow:Starting evaluation at 2017-11-22-11:53:00
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-4000
INFO:tensorflow:Finished evaluation at 2017-11-22-11:53:00
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.966667, average_loss = 0.0733964, global_step = 4000, loss = 2.20189


{'accuracy': 0.96666664,
 'average_loss': 0.073396429,
 'global_step': 4000,
 'loss': 2.2018929}

In [14]:
new_samples = np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": new_samples},
  num_epochs=1,
  shuffle=False)
predictions = classifier.predict(input_fn=predict_input_fn)
for i, p in enumerate(predictions):
  print("Prediction %s: %s" % (i + 1, p["classes"])) #predict 的返回值包含很多内容，需要选择

INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-4000
Prediction 1: [b'1']
Prediction 2: [b'2']
